<a href="https://colab.research.google.com/github/Hwan0130/Bioinformatics_intern/blob/main/Hail_tutorial_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hail을 이용한 전장유전체 분석 기초 실습

고려대학교 인간유전체 연구실 (안준용, 이혜지)

Update: 2022/10/4

Hail을 이용한 전장유전체 데이터 분석 튜토리얼에 오신 여러분을 환영합니다.

본 튜토리얼을 구동하기 위해선, 오른쪽 상단에 있는 Connect 버튼을 누르면 자동으로 구글 코랩이 연결됩니다. 현재 튜토리얼은 구글 코랩에서 사용할 수 있도록 작성되었습니다. 튜토리얼에 사용할 데이터는 다음 링크 - [데이터](https://www.dropbox.com/s/9kquuf894toh98j/data.tar.gz?dl=0)에서 다운 받을 수 있습니다. 다운로드 받은 데이터는 수강생 여러분이 사용하는 구글 드라이브에 업로드하여 주시길 바랍니다. 튜토리얼의 데이터는 [1000 Genome Project의 high coverage depth WGS](https://www.internationalgenome.org/data/) VCF 중 염색체 22번 일부를 임의로 추출하였습니다.

튜토리얼은 자유롭게 재배포 가능합니다. 다만 자료는 제가 공개된 데이터를 임의로 변경하였기 때문에, 꼭 출처를 명시할 필요가 있습니다. 해당 튜토리얼에 대한 피드백은 joonan30@korea.ac.kr 에게 메일을 주세요. 질문은 워크샵 수업시간 외에 따로 받지 않습니다.



## Hail 설정 및 데이터 불러들이기

Hail은 Apache Spark에서 구동되므로, java JRE 버젼 8이나 11이 요구됩니다. 설치 방법은 Hail.is에 나온대로 `pip` 를 이용하여 설치하면 됩니다.

In [ ]:
# Hail 및 필요한 라이브러리 설치하세요. 설치 과정에서 재부팅 (restart) 버튼이 나올수도 있습니다.
!pip install hail

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 왼쪽 패널에서 구글 드라이브 마운트를 선택하여 본인 구글 드라이브와 데이터를 연결해주세요. 연결 후에는 drive 라는 폴더가 나올 것입니다
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# 연결 후에, 필요한 데이터를 현재 작업 디스크에 압축을 풀어주도록 하겠습니다.
!tar -xvzf drive/MyDrive/data.tar.gz

._data
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.dropbox.attrs'
data/
data/._LCR-hs38.bed
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.dropbox.attrs'
data/LCR-hs38.bed
data/._gnomadv3_1_non_neuro_adj_chr22.ht
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.dropbox.attrs'
data/gnomadv3_1_non_neuro_adj_chr22.ht/
data/._.DS_Store
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.apple.FinderInfo'
data/.DS_Store
data/._out.test.chr22.vcf.bgz
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.dropbox.attrs'
data/out.test.chr22.vcf.bgz
data/._1kg_v3_20200731_complete_fam_633.txt
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.dropbox.attrs'
data/1kg_v3_20200731_complete_fam_633.txt
data/._ucsc.encode_cre.hg38.bed
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.dropbox.attrs'
data/ucsc.encode_cre.hg38.bed
data/._1kg_v3_20200731_complete_fam_633.ped
ta

In [ ]:
from bokeh.io import show, output_notebook
from bokeh.layouts import gridplot
import pandas as pd
output_notebook()

In [ ]:
import hail as hl # Hail 라이브러리 로드
hl.init() # Hail 설정이 잘 되었는지 확인합니다

Running on Apache Spark version 3.5.3
SparkUI available at http://9ac059ad4e4f:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.133-4c60fddb171a
LOGGING: writing to /content/hail-20250106-0913-0.2.133-4c60fddb171a.log


### VCF 데이터 불러들이기

VCF 파일을 로드하기 위해 사용되는 함수는 `hl.import_vcf()` 입니다. 이 함수는 VCF 포맷의 데이터를 matrix table 형태로 읽어들이게 됩니다. 우리가 사용하는 데이터는 hg38 버전에서 mapping 되어 있으므로, reference genome 옵션에는 `GRCh38`을 입력해봅시다.
이 외에도 txt, bed, bgen 등의 포맷을 불러들일 수 있는 다양한 함수가 존재합니다. 다음 링크의 hail document를 참고해보세요.

In [ ]:
mt = hl.import_vcf('data/out.test.chr22.vcf.bgz',
                   reference_genome = 'GRCh38',
                   array_elements_required=False)

분석에 들어가기에 앞서, 데이터가 실제로 어떻게 구성되어 있는지 살펴보아야 합니다. 데이터의 전체적인 구조를 살펴보고 싶을 때는 `describe()`를 사용해보세요.

In [ ]:
mt.describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Column fields:
    's': str
----------------------------------------
Row fields:
    'locus': locus<GRCh38>
    'alleles': array<str>
    'rsid': str
    'qual': float64
    'filters': set<str>
    'info': struct {
        AC: array<int32>, 
        AF: array<float64>, 
        AN: int32, 
        BaseQRankSum: float64, 
        ClippingRankSum: float64, 
        DP: int32, 
        DS: bool, 
        END: int32, 
        FS: float64, 
        HaplotypeScore: float64, 
        InbreedingCoeff: float64, 
        MLEAC: array<int32>, 
        MLEAF: array<float64>, 
        MQ: float64, 
        MQ0: int32, 
        MQRankSum: float64, 
        NEGATIVE_TRAIN_SITE: bool, 
        POSITIVE_TRAIN_SITE: bool, 
        QD: float64, 
        RAW_MQ: float64, 
        ReadPosRankSum: float64, 
        SOR: float64, 
        VQSLOD: float64, 
        VariantType: str, 
        culprit: str,


#### Matrix Table의 구조

Matrix table은 총 네 가지 필드로 구분됩니다. 우리의 데이터는 각 필드에 어떤 정보를 담고 있을까요? 이것을 이해하기 위해서는 VCF format에 대해 생각해봐야 합니다.

VCF format은 총 8가지의 variant information (CHROM, POS, ID, REF, ALT, QUAL, FILTER, INFO), 그리고 genotype information (FORMAT, n*sample) 으로 이루어져 있습니다.

![VCF와 Hail matrix table 개요](https://www.dropbox.com/s/kn0hm8oy7ucxqum/vcf_to_hail.png?raw=1)

##### Row 필드

Row 필드 8개의 variant information이 matrix table에 들어가게 됩니다. CHROM, POS 는 'locus'라는 변수로, REF, ALT는 'alleles'라는 변수로 치환되죠. 그리고 이 두 변수는 Row 필드의 key로 지정됩니다.

##### Column 필드

Column 필드는 각 샘플의 메타 정보를 담고 있습니다. VCF Header 중 sample name이 Column 필드로 들어가게 됩니다. 이것은 마찬가지로 Column 필드의 key로 지정됩니다.

##### Entry 필드

Entry 필드는 샘플별 genotype information를 의미합니다. VCF 파일에서의 FORMAT 데이터가 Entry 필드로 들어가는 것이죠.

간단하게 말하면 Column 필드는 sample level information, Row 필드는 variant level information, Entry 필드는 genotype level information 라고 할 수 있습니다.

이번엔 데이터가 어떻게 기록되어 있는지 눈으로 확인할 차례입니다. show() 를 사용하여 데이터의 각 필드를 도표 형태로 출력해보세요.

In [ ]:
mt.cols().show()

2025-01-06 09:15:10.275 Hail: WARN: cols(): Resulting column table is sorted by 'col_key'.
    To preserve matrix table column order, first unkey columns with 'key_cols_by()'


""
s
str
"""HG00096"""
"""HG00097"""
"""HG00099"""
"""HG00100"""
"""HG00101"""
"""HG00102"""
"""HG00103"""
"""HG00105"""


In [ ]:
mt.rows().show()

2023-11-06 22:40:19.691 Hail: INFO: scanning VCF for sortedness...
2023-11-06 22:40:34.217 Hail: INFO: Coerced sorted VCF - no additional import work to do


+----------------+---------------+------+----------+---------------------------------+
| locus          | alleles       | rsid |     qual | filters                         |
+----------------+---------------+------+----------+---------------------------------+
| locus<GRCh38>  | array<str>    | str  |  float64 | set<str>                        |
+----------------+---------------+------+----------+---------------------------------+
| chr22:50197100 | ["T","C"]     | NA   | 1.40e+04 | {}                              |
| chr22:50197103 | ["C","T"]     | NA   | 5.58e+02 | {}                              |
| chr22:50197193 | ["A","C"]     | NA   | 4.70e+02 | {"VQSRTrancheSNP99.80to100.00"} |
| chr22:50197198 | ["C","T"]     | NA   | 1.69e+06 | {}                              |
| chr22:50197199 | ["G","C","A"] | NA   | 3.49e+05 | {}                              |
| chr22:50197200 | ["C","G"]     | NA   | 2.48e+05 | {}                              |
| chr22:50197205 | ["G","A"]     | NA   | 3.53e+04 | {}                              |
| chr22:50197285 | ["G","C"]     | NA   | 9.06e+02 | {}                              |
| chr22:50197300 | ["C","T"]     | NA   | 6.12e+03 | {}                              |
| chr22:50197309 | ["G","C"]     | NA   | 1.09e+03 | {}                              |
+----------------+---------------+------+----------+---------------------------------+

+--------------+---------------------+---------+-------------------+----------------------+
| info.AC      | info.AF             | info.AN | info.BaseQRankSum | info.ClippingRankSum |
+--------------+---------------------+---------+-------------------+----------------------+
| array<int32> | array<float64>      |   int32 |           float64 |              float64 |
+--------------+---------------------+---------+-------------------+----------------------+
| [24]         | [3.75e-03]          |    6404 |          3.64e-01 |             7.92e-01 |
| [1]          | [1.56e-04]          |    6404 |         -1.91e+00 |            -9.06e-01 |
| [15]         | [2.34e-03]          |    6402 |         -1.09e+00 |            -2.01e-01 |
| [2439]       | [3.81e-01]          |    6402 |          1.42e-01 |            -2.40e-02 |
| [534,9]      | [8.34e-02,1.41e-03] |    6404 |          1.56e-01 |             5.50e-02 |
| [287]        | [4.48e-02]          |    6404 |         -7.60e-02 |             4.70e-02 |
| [49]         | [7.65e-03]          |    6404 |          0.00e+00 |            -2.17e-01 |
| [2]          | [3.12e-04]          |    6404 |          1.66e+00 |             5.90e-02 |
| [10]         | [1.56e-03]          |    6404 |          4.70e-01 |             3.28e-01 |
| [2]          | [3.12e-04]          |    6404 |          8.75e-01 |             1.04e+00 |
+--------------+---------------------+---------+-------------------+----------------------+

+---------+---------+----------+----------+---------------------+----------------------+
| info.DP | info.DS | info.END |  info.FS | info.HaplotypeScore | info.InbreedingCoeff |
+---------+---------+----------+----------+---------------------+----------------------+
|   int32 |    bool |    int32 |  float64 |             float64 |              float64 |
+---------+---------+----------+----------+---------------------+----------------------+
|  121389 |   False |       NA | 2.44e+00 |                  NA |                   NA |
|  121310 |   False |       NA | 1.12e+00 |                  NA |            -2.00e-04 |
|  123891 |   False |       NA | 2.61e+01 |                  NA |            -9.40e-03 |
|  135837 |   False |       NA | 1.13e+00 |                  NA |             3.30e-03 |
|  123182 |   False |       NA | 1.23e+01 |                  NA |             1.41e-01 |
|  124771 |   False |       NA | 1.11e+00 |                  NA |             1.25e-01 |
|  122821 |   False |       NA | 6.88e+00 |                  NA |            -5.30e-03 |
|  121052 |   False |       NA | 8.56e-01 |                  NA | 

In [ ]:
mt.entries().show()

2023-11-06 22:40:38.215 Hail: WARN: entries(): Resulting entries table is sorted by '(row_key, col_key)'.
    To preserve row-major matrix table order, first unkey columns with 'key_cols_by()'


+----------------+------------+------+----------+----------+--------------+----------------+
| locus          | alleles    | rsid |     qual | filters  | info.AC      | info.AF        |
+----------------+------------+------+----------+----------+--------------+----------------+
| locus<GRCh38>  | array<str> | str  |  float64 | set<str> | array<int32> | array<float64> |
+----------------+------------+------+----------+----------+--------------+----------------+
| chr22:50197100 | ["T","C"]  | NA   | 1.40e+04 | {}       | [24]         | [3.75e-03]     |
| chr22:50197100 | ["T","C"]  | NA   | 1.40e+04 | {}       | [24]         | [3.75e-03]     |
| chr22:50197100 | ["T","C"]  | NA   | 1.40e+04 | {}       | [24]         | [3.75e-03]     |
| chr22:50197100 | ["T","C"]  | NA   | 1.40e+04 | {}       | [24]         | [3.75e-03]     |
| chr22:50197100 | ["T","C"]  | NA   | 1.40e+04 | {}       | [24]         | [3.75e-03]     |
| chr22:50197100 | ["T","C"]  | NA   | 1.40e+04 | {}       | [24]         | [3.75e-03]     |
| chr22:50197100 | ["T","C"]  | NA   | 1.40e+04 | {}       | [24]         | [3.75e-03]     |
| chr22:50197100 | ["T","C"]  | NA   | 1.40e+04 | {}       | [24]         | [3.75e-03]     |
| chr22:50197100 | ["T","C"]  | NA   | 1.40e+04 | {}       | [24]         | [3.75e-03]     |
| chr22:50197100 | ["T","C"]  | NA   | 1.40e+04 | {}       | [24]         | [3.75e-03]     |
+----------------+------------+------+----------+----------+--------------+----------------+

+---------+-------------------+----------------------+---------+---------+----------+----------+
| info.AN | info.BaseQRankSum | info.ClippingRankSum | info.DP | info.DS | info.END |  info.FS |
+---------+-------------------+----------------------+---------+---------+----------+----------+
|   int32 |           float64 |              float64 |   int32 |    bool |    int32 |  float64 |
+---------+-------------------+----------------------+---------+---------+----------+----------+
|    6404 |          3.64e-01 |             7.92e-01 |  121389 |   False |       NA | 2.44e+00 |
|    6404 |          3.64e-01 |             7.92e-01 |  121389 |   False |       NA | 2.44e+00 |
|    6404 |          3.64e-01 |             7.92e-01 |  121389 |   False |       NA | 2.44e+00 |
|    6404 |          3.64e-01 |             7.92e-01 |  121389 |   False |       NA | 2.44e+00 |
|    6404 |          3.64e-01 |             7.92e-01 |  121389 |   False |       NA | 2.44e+00 |
|    6404 |          3.64e-01 |             7.92e-01 |  121389 |   False |       NA | 2.44e+00 |
|    6404 |          3.64e-01 |             7.92e-01 |  121389 |   False |       NA | 2.44e+00 |
|    6404 |          3.64e-01 |             7.92e-01 |  121389 |   False |       NA | 2.44e+00 |
|    6404 |          3.64e-01 |             7.92e-01 |  121389 |   False |       NA | 2.44e+00 |
|    6404 |          3.64e-01 |             7.92e-01 |  121389 |   False |       NA | 2.44e+00 |
+---------+-------------------+----------------------+---------+---------+----------+----------+

+---------------------+----------------------+--------------+----------------+----------+----------+
| info.HaplotypeScore | info.InbreedingCoeff | info.MLEAC   | info.MLEAF     |  info.MQ | info.MQ0 |
+---------------------+----------------------+--------------+----------------+----------+----------+
|             float64 |              float64 | array<int32> | array<float64> |  float64 |    int32 |
+---------------------+----------------------+--------------+----------------+----------+----------+
|                  NA |                   NA | [24]         | [3.75e-03]     | 6.00e+01 |        0 |
|                  NA |                   NA | [24]         | [3.75e-03]     | 6.00e+01 |        0 |
|                  NA |                   NA | [24]         | [3.75e-03]     | 6.00e+01 |        0 |
|                  NA |                   NA | [24]         | [3.75e-03]     | 6.00e+01 |        0 |
|                  NA |                   NA | [2

### 샘플 정보 불러들이기


Column 필드는 sample level information, 즉 샘플 개개인의 메타 정보를 담고 있습니다. 하지만 VCF 파일에서 얻을 수 있는 샘플 정보는 이름밖에 없죠. 따라서 샘플의 Pedigree 정보를 알 수 있는 '.ped' 포맷의 파일을 불러들일 것입니다. Ped 파일을 불러들일 때는 `import_fam()`를 사용합니다. Ped file format은 Pedigree, Sex, Phenotype 정보를 담고 있습니다. 자세한 정보는 다음 [링크](https://gatk.broadinstitute.org/hc/en-us/articles/360035531972-PED-Pedigree-format)를 참고하세요. 이 가족 데이터를 이용해서 inherited variant (유전이 되는 유전 변이)와 germline de novo variant (부모의 생식세포에서 발생하여 자녀에게만 나타나는 변이)를 확인하려고 합니다.

In [ ]:
fam = hl.import_fam('data/1kg_v3_20200731_complete_fam_633.ped')
fam.describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Row fields:
    'id': str 
    'fam_id': str 
    'pat_id': str 
    'mat_id': str 
    'is_female': bool 
    'is_case': bool 
----------------------------------------
Key: ['id']
----------------------------------------


이 외에도 샘플 정보를 추가하고 싶으면, 테이블 형태(txt, tsv 등) 파일을 불러들여 추가할 수 있습니다. 우리는 샘플이 자녀인지, 부모인지를 구분하기 위해 추가적인 정보를 하나 더 불러오도록 하겠습니다. `import_table()`는 txt, tsv 등의 파일을 hail table 형태로 읽어들입니다. 데이터 불러들일 때 'key' 옵션을 통해 특정 변수를 key로 지정할 수 있습니다. Hail 에서 VCF, bed, ped 등 고정된 형식의 데이터를 불러올 때는 자동적으로 key가 지정됩니다. 하지만 두 번째로 불러들인 txt 파일은 그렇지 않기 때문에 옵션을 통해 샘플 id를 key로 지정해줘야 합니다.

In [ ]:
info = hl.import_table('data/1kg_v3_20200731_complete_fam_633.txt',
                       key = 'Individual ID',
                       types = {'height' : hl.tfloat, 'NVIQ' : hl.tfloat})
info.describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Row fields:
    'FID': str 
    'Individual ID': str 
    'Paternal ID': str 
    'Maternal ID': str 
    'Gender': str 
    'Phenotype': str 
    'ROLE': str 
    'height': float64 
    'NVIQ': float64 
    'Population': str 
----------------------------------------
Key: ['Individual ID']
----------------------------------------


2023-11-06 22:40:48.768 Hail: INFO: Reading table without type imputation
  Loading field 'FID' as type str (not specified)
  Loading field 'Individual ID' as type str (not specified)
  Loading field 'Paternal ID' as type str (not specified)
  Loading field 'Maternal ID' as type str (not specified)
  Loading field 'Gender' as type str (not specified)
  Loading field 'Phenotype' as type str (not specified)
  Loading field 'ROLE' as type str (not specified)
  Loading field 'height' as type float64 (user-supplied)
  Loading field 'NVIQ' as type float64 (user-supplied)
  Loading field 'Population' as type str (not specified)


불러온 샘플 정보를 Column 필드에 annotate 할 차례입니다. `annotate_cols()` 함수를 사용해보세요.

In [ ]:
# Annotate sample & pedigree information to variants data
mt = mt.annotate_cols(fam = fam[mt.s].fam_id,
                     pat_id = fam[mt.s].pat_id,
                     mat_id = fam[mt.s].mat_id,
                     isFemale = fam[mt.s].is_female,
                     phenotype = fam[mt.s].is_case,
                     ROLE = info[mt.s].ROLE,
                     population = info[mt.s].Population,
                     height = info[mt.s].height,
                     NVIQ = info[mt.s].NVIQ)

In [ ]:
mt.describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Column fields:
    's': str
    'fam': str
    'pat_id': str
    'mat_id': str
    'isFemale': bool
    'phenotype': bool
    'ROLE': str
    'population': str
    'height': float64
    'NVIQ': float64
----------------------------------------
Row fields:
    'locus': locus<GRCh38>
    'alleles': array<str>
    'rsid': str
    'qual': float64
    'filters': set<str>
    'info': struct {
        AC: array<int32>, 
        AF: array<float64>, 
        AN: int32, 
        BaseQRankSum: float64, 
        ClippingRankSum: float64, 
        DP: int32, 
        DS: bool, 
        END: int32, 
        FS: float64, 
        HaplotypeScore: float64, 
        InbreedingCoeff: float64, 
        MLEAC: array<int32>, 
        MLEAF: array<float64>, 
        MQ: float64, 
        MQ0: int32, 
        MQRankSum: float64, 
        NEGATIVE_TRAIN_SITE: bool, 
        POSITIVE_TRAIN_SITE: bool, 
     

이번엔 샘플 정보에 따라 matrix table에 존재하는 샘플들을 추출해봅시다

In [ ]:
print("Before filtering samples, (# of variants, # of samples) = ", mt.count())
mt_chs = mt.filter_cols(mt.population=='CHS', keep=True) # filter out samples that are not in ped file
print("After filtering samples, (# of variants, # of samples) = ", mt_chs.count())
# mt.cols().show()

Before filtering samples, (# of variants, # of samples) =  (16515, 3202)
After filtering samples, (# of variants, # of samples) =  (16515, 160)


### Function annotation 을 추가하기

전장유전체 분석에서 찾은 유전변이가 특정 기능을 담당하는 지역에서 발생할 수 있습니다. 전장유전체는 noncoding genome에서도 변이를 찾을 수 있기 때문에, 유전자 발현과정에서 전사조절인자가 붙는 지역을 지정하면, 그 지역에서 발생한 변이를 추출할 수 있습니다. 아래는 ENCODE 프로젝트에서 chip-seq을 통해 찾은 다양한 Cis-Regulatory Elements (CRE)가 붙는 유전체 지역에 대한 정보를 bed file 형태로 UCSC에서 다운로드 받았습니다 ([링크](https://genome.ucsc.edu/cgi-bin/hgTrackUi?hgsid=1259223423_dDFJRqzUKODz4m3GycQ5RA0apAYw&db=hg38&c=chr22&g=encodeCcreCombined)).

```
bigBedToBed http://hgdownload.soe.ucsc.edu/gbdb/hg38/encode3/ccre/encodeCcreCombined.bb -chrom=chr21 -start=0 -end=100000000 stdout | cut -f1-4 > ucsc.encode_cre.hg38.bed
```

In [ ]:
tb_encode_cre = hl.import_bed('data/ucsc.encode_cre.hg38.bed', reference_genome='GRCh38')
tb_encode_cre.describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Row fields:
    'interval': interval<locus<GRCh38>> 
    'target': str 
----------------------------------------
Key: ['interval']
----------------------------------------


2023-11-06 22:41:15.378 Hail: INFO: Reading table without type imputation
  Loading field 'f0' as type str (user-supplied)
  Loading field 'f1' as type int32 (user-supplied)
  Loading field 'f2' as type int32 (user-supplied)
  Loading field 'f3' as type str (user-supplied)


## Hail 기본 문법을 이용한 데이터 둘러보기

Hail 기본 문법에 익숙해지기 위해, 불러드린 자료들의 정보를 확인해봅시다.

### 유전변이 정보 확인하기

GATK 파이프라인에서 만들어진 VCF는 각 샘플의 variant와 genotype 정보를 갖고 있습니다. Variant는 특정 locus에서 염기서열이 어떻게 변화했는지에 대한 것이고, genotype은 allele에 대한 정보를 갖고 있습니다.

또한, VCF에는 variant calling에 대한 품질지표(quality metrics)를 variant과 genotype에 대하여 제공합니다.

1. Variant 수준의 품질 지표는 해당 VCF에 존재하는 샘플들에게 공통적으로 산출된 지표이고, locus 수준의 지표라고 할 수 있습니다. INFO 컬럼에 저장되어 있습니다.
2. Genotype 수준의 품질 지표는 각 locus에서 발견된 variant에 대하여 샘플이 갖는 지표를 말합니다. 각 샘플 컬럼에 저장되어 있고, FORMAT 컬럼에 저장된 지표들을 따릅니다. 예를들어, FORMAT에 GT:AD:PL 순서로 나온다면, 샘플 컬럼에도 동일한 정보(0/1:30,30:100,0,1000)로 저장되어 있습니다.

먼저 특정 locus에서 variant를 추출해봅시다.

In [ ]:
locus = hl.locus(contig = "chr22", pos = 50197928, reference_genome = 'GRCh38')
mt.filter_rows(mt.locus == locus).show()

,
locus,alleles
locus<GRCh38>,array<str>
chr22:50197928,"[""C"",""T""]"


자, 염색체 22번의 `50197928` 번째 위치에  나타난 변이를 보면 `["C", "T"]`와 같은 list가 존재합니다. 처음의 `C`는 reference allele에 해당하고, `T`는 alternative allele에 해당합니다. 우리는 `T`를 `변이(variant)`라고 부릅니다.

변이는 "염기서열 상의 변화"이고 현재 사용하는 VCF에 하나의 row에 기록됩니다.

유전형(genotype)은 이 VCF에 포함된 샘플들에 개별적으로 기록됩니다. 유전형은 3가지 경우로 구분됩니다.


이번에는 특정 지역에서 발생하는 variant를 추출해봅시다. 우리는 염색체 22q13.3에 위치한 SHANK3 유전자에 발생한 변이들을 보고 싶습니다. 이 유전자는 뇌기능에 중요한 유전자이면서, 변이가 발생하면 Phelan-McDermid syndrome 등을 일으킨다고 잘 알려져있죠. 그러면 SHANK3 위치(`chr22:50674642-50731312`)를 이용해서 변이를 추출해보겠습니다.

In [ ]:
locus = hl.locus_interval("chr22", 50674642, 50731312, reference_genome='GRCh38')
hl.filter_intervals(mt, [locus]).show()

,
locus,alleles
locus<GRCh38>,array<str>


일반적으로 인간 유전체의 유전변이는 2개의 allele로 구성된 3개의 유전형(genotype)을 갖습니다. 이것을 bi-allelic variant라고 부릅니다. 하나의 allele은 참조유전체(reference genome)과 동일한 것이므로 reference allele이라 부르고, 다른 하나는 변이에 의해 생기는 alternative allele (혹은 variant라고 allele)이라고 부릅니다.

그러나 종종 예외도 존재합니다. 바로 multi-allelic variant 입니다. 위 테이블을 보시면, 두가지 이상의 allele이 나오는 경우가 있습니다. `50674782` 위치에서 발생한 변이를 보면, `["C","A","G"]`이와 같이 3개의 allele이 존재합니다. 우리는 이걸 `multi-allelic variant`라고 부릅니다. 같은 locus에 두개 이상의 alternative allele이 나오는 경우이죠.

전장유전체가 보편화 된 초기(2014년 쯤)에 `multi-allelic variant`는 큰 이슈였습니다. 그 시기쯤 GATK에서 joint genotyping module을 도입했고, multi-sample VCF가 만들어지기 시작했습니다. Multi-sample VCF를 만드는 joint genotyping은 하나의 샘플만 genotyping을 하는 individual genotyping보다 유리한 점이 많습니다.

1. 각 variant에 대하여 샘플간의 유전형(genotype) 정보를 확보할 수 있습니다. 특히 reference homozygous genotype에 대한 정보와 그 genotype에 대한 품질지표를 확인할 수 있습니다.
2. Variant calling에 포함된 샘플들의 품질지표를 사용하여 high quality variant를 정의하기 용이합니다.
3. 가족 데이터를 사용할 경우 phasing이 용이합니다.

자세한 정보는 아래 논문을 확인하세요 ([Zook et al. 2014](https://www.nature.com/articles/nbt.2835), [Koboldt 2020](https://genomemedicine.biomedcentral.com/articles/10.1186/s13073-020-00791-w)).

Multi-sample VCF에서는 `ALT` 컬럼에 표시되는 allele이 두개 이상인 multi-allelic variant가 등장합니다. VCF에는 어떻게 표현이 되었는지 보시죠.

```
chr22	50674782	.	C	A,G

```

이 경우엔 일부 샘플은 `A` allele을 갖고 있고, 일부 샘플은 `G`를 갖고 있습니다. 두가지 variant가 등장을 했습니다. `C>A` 혹은 `C>G` 변이이지요. 만약 individual genotyping을 해서 하나의 샘플만 보았다면, 관찰되지 않을 경우입니다.

혹은 이러한 경우도 존재합니다. `50675375`번째 위치를 보면 `["G","*","T"]`라고 `* (STAR)`가 표시되어 있습니다. 이것은 alternative allele은 아닙니다. 이것은 같은 VCF에 있는 어떤 샘플이 deletion을 갖고 있는데, 그 지역에 다른 샘플이 SNV를 갖는 경우, 표시할 방법이 없기 때문에 위와 같이 표기를 합니다.

Multi-allelic variant가 나오면, 각 variant에 대한 정보나 품질지표가 붙기 때문에 하나의 row에서 list나 array로 변환하기가 다소 번거롭습니다. 이러한 이유로 multi-allelic variant를 bi-allelic variant로 변형하는 작업이 필요합니다. Hail은 이 작업을 매우 간단하게 시행합니다.


In [ ]:
print("--> Before split variants with multi allele, the number of variants is :", mt.rows().count())

--> Before split variants with multi allele, the number of variants is : 16515


현재 우리 데이터는 총 16,515개의 variant가 존재합니다. 이제 `split_multi()`라는 함수를 사용하여, multi-allelic variant를 분리(split)해봅시다.

분리가 되고나면, 아래와 같이 변환이 됩니다.

```
chr22	50674782	.	C	A
chr22	50674782	.	C	G
```

In [ ]:
mt = hl.split_multi(mt)

print("--> After split variants with multi allele, the number of variants is :", mt.rows().count())

--> After split variants with multi allele, the number of variants is : 17898


Multi-allelic variant를 bi-allelic variant로 분리하고 나니, 총 17,898개의 변이로 변환됩니다.

> 리빙포인트: 거의 모든 분석은 bi-allelic variant를 요구합니다. 따라서, `split_multi()` 함수를 생활화 합시다.

In [ ]:
# 이번에는 CRE 영역에서 발생한 변이들을 추출해봅시다
mt.filter_rows(hl.is_defined(tb_encode_cre[mt.locus])).show()

# 혹은 새로운 matrix table로 저장을 해봅시다
mt_cre = mt.filter_rows(hl.is_defined(tb_encode_cre[mt.locus]))

2023-11-06 22:42:18.723 Hail: INFO: Coerced sorted dataset
2023-11-06 22:42:19.805 Hail: INFO: Coerced sorted dataset


,
locus,alleles
locus<GRCh38>,array<str>
chr22:50197928,"[""C"",""T""]"
chr22:50197932,"[""G"",""A""]"
chr22:50197962,"[""G"",""A""]"
chr22:50197973,"[""C"",""T""]"
chr22:50197984,"[""G"",""A""]"
chr22:50198000,"[""C"",""T""]"
chr22:50198004,"[""C"",""T""]"
chr22:50198005,"[""G"",""A""]"


이번에는 변이를 SNV(혹은 SNP)와 indel로 구분해봅시다.

SNV는 염기의 숫자가 줄어들지 않고, 하나의 염기가 바뀌는 변이를 말하고, indel은 insertion과 deletion을 합친 약어로서 1개 이상 그리고 50개 미만의 염기서열이 변경되는 경우를 말합니다 (참고 여기서 몇개 미만에 대한 기준은 연구들마다 상이합니다).

먼저 SNV의 숫자를 세어봅시다.

In [ ]:
print("--> The number of SNVs is :", mt.filter_rows(hl.is_snp(mt.alleles[0], mt.alleles[1])).count())

--> The number of SNVs is : (14933, 3202)


이번에는 indel의 숫자를 세어보죠.

In [ ]:
print("--> The number of SNVs is :", mt.filter_rows(hl.is_indel(mt.alleles[0], mt.alleles[1])).count())

--> The number of SNVs is : (2965, 3202)


VCF 유전변이에 대한 정보나 품질지표가 `INFO` 컬럼에 포함되어 있습니다. 유전변이가 발생한 지점의 read depth (DP), strand bias (FS 혹은 SOR), mapping alignment quality (MQ) 등의 품질지표가 있고, locus에 있는 alternative allele의 숫자 (AC)나 확인된 allele의 숫자가 있습니다.

먼저 Hail을 이용하여, `info`에 어떠한 지표들이 있는지 확인하고, 몇가지 지표들의 분포를 확인해봅시다.

In [ ]:
mt.describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Column fields:
    's': str
    'fam': str
    'pat_id': str
    'mat_id': str
    'isFemale': bool
    'phenotype': bool
    'ROLE': str
    'population': str
    'height': float64
    'NVIQ': float64
----------------------------------------
Row fields:
    'locus': locus<GRCh38>
    'alleles': array<str>
    'rsid': str
    'qual': float64
    'filters': set<str>
    'info': struct {
        AC: array<int32>, 
        AF: array<float64>, 
        AN: int32, 
        BaseQRankSum: float64, 
        ClippingRankSum: float64, 
        DP: int32, 
        DS: bool, 
        END: int32, 
        FS: float64, 
        HaplotypeScore: float64, 
        InbreedingCoeff: float64, 
        MLEAC: array<int32>, 
        MLEAF: array<float64>, 
        MQ: float64, 
        MQ0: int32, 
        MQRankSum: float64, 
        NEGATIVE_TRAIN_SITE: bool, 
        POSITIVE_TRAIN_SITE: bool, 
     

In [ ]:
p = hl.plot.histogram(mt.info.DP, title = 'DP')
show(p)

In [ ]:
p = hl.plot.histogram(mt.info.MQ, title = 'MQ')
show(p)

In [ ]:
p = hl.plot.histogram(mt.info.SOR, title = 'SOR')
show(p)

In [ ]:
p = hl.plot.histogram(mt.info.QD, title = 'QD')
show(p)

이번엔 AC와 AN의 정보를 확인해봅시다. AC와 AN은 전장유전체 정도관리 (quality control)에 매우 중요한 지표입니다. 여러명의 샘플을 joint genotyping 하는 경우에, 특정 부위는 variant calling이나 sequencing이 잘 안되는 경우가 발생합니다. 그러한 loci는 AN이 낮게 나타나고, 오직 소수의 샘플에서만 유전형이 확인됩니다. 이러한 변이는 연관성 검정에서 사용하기가 어렵습니다. 인간 유전체는 AN이 샘플수 * 2 (allele의 숫자)이 상염색체에선 최대값입니다.

AC는 allele의 빈도를 구할때 중요한 지표입니다. VCF 상에서 AF를 정의하는 것은 AC/AN으로 합니다.

In [ ]:
p = hl.plot.histogram(mt.info.AC[0], title = 'AC') # AC는 array로 존재하므로, 첫번째 아이템을 선택합니다
show(p)

In [ ]:
p = hl.plot.histogram(mt.info.AN, title = 'AN')
show(p)

SNV와 Indel은 여러가지 이유로 품질지표에서 다른 분포를 갖습니다. 몇가지 지표를 보고 SNV와 indel의 차이점을 살펴봅시다. 먼저 SNV와 indel로 matrix table을 나누겠습니다.

In [ ]:
mt_SNV = mt.filter_rows(hl.is_snp(mt.alleles[0], mt.alleles[1]))
mt_Indel = mt.filter_rows(hl.is_indel(mt.alleles[0], mt.alleles[1]))

In [ ]:
p = hl.plot.histogram(mt_SNV.info.MQ, title = 'MQ - SNV')
show(p)

In [ ]:
p = hl.plot.histogram(mt_Indel.info.MQ, title = 'MQ - Indel')
show(p)

VCF에는 variant calling 알고리즘에 따라 분류된 quality filter가 존재합니다. 우리가 사용한 VCF는 GATK의 VQSR 모델에 따라 `PASS` filter를 제공합니다. 향후 분석을 위해 `PASS` variant만 사용하도록 추출합시다.

In [ ]:
print("--> Before remove variants with VSQR pass failure, the number of variants is :", mt.rows().count())
mt = mt.filter_rows(hl.len(mt.filters) == 0)
print("--> After remove variants with VSQR pass failure, the number of variants is :", mt.rows().count())

--> Before remove variants with VSQR pass failure, the number of variants is : 17898
--> After remove variants with VSQR pass failure, the number of variants is : 16188


### 유전형 정보 확인하기

유전변이(variant)와 다르게 유전형(genotype)은 개별 샘플에 나타나는 정보에 해당합니다.

**1. Reference homozygous call**

이에 속하는 샘플은 alternative allele이 없고 DNA 두쌍이 모두 reference allele을 갖습니다. 이런 경우를 `0/0`으로 표시합니다.

**2. Heterozygous variant**

이에 속하는 샘플은 DNA 두쌍 중 하나가 alternative allele을 갖고, 다른 하나는 reference allele을 갖습니다. 이런 경우를 `0/1`으로 표시합니다.

**3. Homozygous variant**

이에 속하는 샘플은 DNA 두쌍 중 하나가 alternative allele을 갖습니다. 이런 경우를 `1/1`으로 표시합니다.


아래 FORMAT에 보면, `GT:AB:AD:DP:GQ:PL` 이라는 순서로 정보가 존재하고, 첫번째 샘플 (HG00096)은 `0/0:.:36,0:36:99:0,99,1301`을 갖습니다.


```
#CHROM	POS	ID	REF	ALT	QUAL	FILTER	INFO	FORMAT	HG00096	HG00097	HG00099
chr22	50197100	.	T	C	13997.6	PASS	AC=24;AF=0.00374766;AN=6404;BaseQRankSum=0.364;ClippingRankSum=0.792;DP=121389;FS=2.436;MLEAC=24;MLEAF=0.003748;MQ=60;MQ0=0;MQRankSum=0.217;QD=13.17;ReadPosRankSum=0.892;SOR=0.862;VQSLOD=11.6;culprit=MQ;AN_EUR=1266;AN_EAS=1170;AN_AMR=980;AN_SAS=1202;AN_AFR=1786;AF_EUR=0;AF_EAS=0;AF_AMR=0.00408163;AF_SAS=0;AF_AFR=0.0111982;AC_EUR=0;AC_EAS=0;AC_AMR=4;AC_SAS=0;AC_AFR=20;AC_Het_EUR=0;AC_Het_EAS=0;AC_Het_AMR=4;AC_Het_SAS=0;AC_Het_AFR=20;AC_Het=24;AC_Hom_EUR=0;AC_Hom_EAS=0;AC_Hom_AMR=0;AC_Hom_SAS=0;AC_Hom_AFR=0;AC_Hom=0;HWE_EUR=1;ExcHet_EUR=1;HWE_EAS=1;ExcHet_EAS=1;HWE_AMR=1;ExcHet_AMR=0.993874;HWE_SAS=1;ExcHet_SAS=1;HWE_AFR=1;ExcHet_AFR=0.898023;HWE=1;ExcHet=0.957665;ME=0;AN_EUR_unrel=1006;AN_EAS_unrel=1008;AN_AMR_unrel=694;AN_SAS_unrel=978;AN_AFR_unrel=1322;AF_EUR_unrel=0;AF_EAS_unrel=0;AF_AMR_unrel=0.00288184;AF_SAS_unrel=0;AF_AFR_unrel=0.0121029;AC_EUR_unrel=0;AC_EAS_unrel=0;AC_AMR_unrel=2;AC_SAS_unrel=0;AC_AFR_unrel=16;AC_Het_EUR_unrel=0;AC_Het_EAS_unrel=0;AC_Het_AMR_unrel=2;AC_Het_SAS_unrel=0;AC_Het_AFR_unrel=16;AC_Hom_EUR_unrel=0;AC_Hom_EAS_unrel=0;AC_Hom_AMR_unrel=0;AC_Hom_SAS_unrel=0;AC_Hom_AFR_unrel=0;CSQ=C|upstream_gene_variant|MODIFIER|SELENOO|ENSG00000073169|Transcript|ENST00000380903|protein_coding|||||||||||3911|1||HGNC|HGNC:30395	GT:AB:AD:DP:GQ:PL	0/0:.:36,0:36:99:0,99,1301	0/0:.:39,0:39:99:0,105,1260	0/0:.:53,0:53:99:0,120,1800
```

AB는 allelic balance를 의미하고, reference allele과 alternative allele을 갖는 비율을 의미합니다. 즉 heterozygous variant이면, 50%의 비율로 AB가 형성되어야 합니다. AB는 VCF에 따라 homozygous 정보를 제공하지 않는 경우도 있습니다. 그래서 AD를 이용하여, AB를 계산합니다. AD는 array로 제공되는데, 첫번째 위치엔 reference allele를 갖는 read의 수, 두번째 위치엔 alternative allele를 갖는 read의 수가 존재합니다.

In [ ]:
mt = mt.annotate_entries(AB = mt.AD[1] / hl.sum(mt.AD))

이제 heterozygous variant의 AB를 확인해봅시다.

In [ ]:
p = hl.plot.histogram(mt.filter_entries(mt.GT.is_het(), keep=True).AB, title = 'AB - heterozygous variant')
show(p)

Heterozygous variant 임에도 불구하고, 50%에서 넓은 분포가 나타납니다. 여러가지 이유가 있습니다. 1) sequence read가 allele을 갖는것은 stochastic process임으로 확률적으로 분포를 갖습니다. 2) read 내에 존재하는 GC 염기서열의 분포에 따라 시퀀싱이나 PCR에 영향을 줍니다. 3) 현재 matrix table은 low quality variant를 포함하고 있고 이에 따라 낮은 AB가 나타나기도 합니다.

In [ ]:
p = hl.plot.histogram(mt.filter_entries(mt.GT.is_hom_var(), keep=True).AB, title = 'AB - homozygous variant')
show(p)

Homozygous variant라고 예측된 변이들도 AB가 1보다 한참 떨어진 변이들이 관찰됩니다. 위에서 본 heterozygous variant와 동일한 이유입니다. 다시 말하자면, 우리가 전장유전체 분석을 하기 전에 high-quality variant 선별을 위해 상당히 꼼꼼하게 필터링을 수행해야합니다.

### 샘플 정보 확인하기

우리는 위에 VCF를 matrix table로 불러들인후, 샘플 정보를 추가하였습니다. 추가된 정보를 먼저 확인해봅시다. `cols`에 저장된 정보를 불러옵니다.

In [ ]:
mt.cols().show()

,,,,,,,,,
s,fam,pat_id,mat_id,isFemale,phenotype,ROLE,population,height,NVIQ
str,str,str,str,bool,bool,str,str,float64,float64
"""HG00096""",NA,NA,NA,NA,NA,NA,NA,NA,NA
"""HG00097""",NA,NA,NA,NA,NA,NA,NA,NA,NA
"""HG00099""",NA,NA,NA,NA,NA,NA,NA,NA,NA
"""HG00100""",NA,NA,NA,NA,NA,NA,NA,NA,NA
"""HG00101""",NA,NA,NA,NA,NA,NA,NA,NA,NA
"""HG00102""",NA,NA,NA,NA,NA,NA,NA,NA,NA
"""HG00103""",NA,NA,NA,NA,NA,NA,NA,NA,NA
"""HG00105""",NA,NA,NA,NA,NA,NA,NA,NA,NA


먼저 가족별로 몇개의 샘플이 있는지를 확인해봅시다.

In [ ]:
mt.aggregate_cols(
    hl.struct(n_families=hl.agg.counter(mt.fam))
)

Struct(n_families={'fam1': 3, 'fam10': 3, 'fam100': 3, 'fam101': 3, 'fam102': 3, 'fam103': 3, 'fam104': 3, 'fam105': 3, 'fam106': 3, 'fam107': 2, 'fam108': 1, 'fam109': 2, 'fam11': 3, 'fam110': 3, 'fam111': 3, 'fam112': 3, 'fam113': 3, 'fam114': 3, 'fam115': 3, 'fam116': 3, 'fam117': 2, 'fam118': 3, 'fam119': 3, 'fam12': 3, 'fam120': 3, 'fam121': 3, 'fam122': 3, 'fam123': 2, 'fam124': 2, 'fam125': 2, 'fam126': 2, 'fam127': 2, 'fam128': 3, 'fam129': 3, 'fam13': 3, 'fam130': 2, 'fam132': 3, 'fam133': 3, 'fam134': 3, 'fam135': 1, 'fam137': 2, 'fam138': 3, 'fam139': 3, 'fam14': 3, 'fam140': 3, 'fam141': 3, 'fam142': 3, 'fam143': 3, 'fam144': 3, 'fam145': 3, 'fam146': 3, 'fam147': 3, 'fam148': 3, 'fam149': 3, 'fam15': 3, 'fam150': 3, 'fam151': 3, 'fam152': 3, 'fam153': 3, 'fam154': 3, 'fam155': 3, 'fam156': 3, 'fam157': 3, 'fam158': 3, 'fam159': 3, 'fam16': 3, 'fam160': 3, 'fam161': 3, 'fam162': 3, 'fam163': 3, 'fam164': 3, 'fam165': 3, 'fam166': 3, 'fam167': 3, 'fam168': 3, 'fam169': 3, 'f

일부는 3인가족으로 보이고, 일부는 그렇지 않습니다. 3인가족은 부모와 자녀를 의미합니다. 이러한 가족을 추출해봅시다.

In [ ]:
print("Before filtering trios, (# of variants, # of samples) = ", mt.count())
ID = hl.literal(mt.aggregate_cols(hl.agg.collect(mt.s)))
fid = mt.filter_cols((ID.contains(mt.pat_id)) & (ID.contains(mt.mat_id))) # 부모의 ID가 모두 존재

fid = hl.literal(fid.aggregate_cols(hl.agg.collect(fid.fam)))
mt = mt.filter_cols(fid.contains(mt.fam))

print("After filtering trios, (# of variants, # of samples) = ", mt.count())

Before filtering trios, (# of variants, # of samples) =  (16188, 3202)
After filtering trios, (# of variants, # of samples) =  (16188, 1784)


이제 trio 가족들만 남았습니다.

이번에는 우리 데이터에 있는 여성 샘플의 비율을 계산해봅시다. Hail은 fraction을 직접 측정할 수도 있고, 아니면 직접 산출을 할 수도 있습니다.

In [ ]:
mt.aggregate_cols(
    hl.struct(
        fraction_female=hl.agg.fraction(mt.isFemale),
        case_ratio=hl.agg.count_where(mt.isFemale) / hl.agg.count())
)

Struct(fraction_female=0.4938340807174888, case_ratio=0.4938340807174888)

샘플에 저장된 continuous variable도 계산을 할 수 있습니다. 현재 튜토리얼 데이터에는 임의로 키(height)를 추가했습니다. 이를 활용하여 평균 값을 구해보죠.

In [ ]:
mt.aggregate_cols(
    hl.struct(
        height_mean=hl.agg.mean(mt.height))
)

Struct(height_mean=167.57430886673595)

평균키는 167cm로 확인이 되었습니다.

이번에는 키의 분포를 시각화해봅시다.

In [ ]:
p = hl.plot.histogram(mt.height, title = 'Heights')
show(p)

## 품질지표 확인 및 정도관리

### 샘플

전장유전체 분석을 하기 전에, 분석에 사용하는 모든 샘플이 균일하게 시퀀싱이 되었는지 확인할 필요가 있습니다. 먼저 각 샘플이 갖고 있는 변이에 관한 지표를 샘플별로 살펴보겠습니다. 많은 경우 SNV와 indel은 다른 품질 지표를 갖기 때문에, 둘을 구별하여 분석을 해보도록 하겠습니다.

`sample_qc()` 라는 함수를 사용하면, 품질지표에 대한 정보를 계산해줍니다.

In [ ]:
mt_SNV = hl.sample_qc(mt_SNV)
mt_Indel = hl.sample_qc(mt_Indel)

mt_SNV.cols().describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Row fields:
    's': str 
    'fam': str 
    'pat_id': str 
    'mat_id': str 
    'isFemale': bool 
    'phenotype': bool 
    'ROLE': str 
    'population': str 
    'height': float64 
    'NVIQ': float64 
    'sample_qc': struct {
        dp_stats: struct {
            mean: float64, 
            stdev: float64, 
            min: float64, 
            max: float64
        }, 
        gq_stats: struct {
            mean: float64, 
            stdev: float64, 
            min: float64, 
            max: float64
        }, 
        call_rate: float64, 
        n_called: int64, 
        n_not_called: int64, 
        n_filtered: int64, 
        n_hom_ref: int64, 
        n_het: int64, 
        n_hom_var: int64, 
        n_non_ref: int64, 
        n_singleton: int64, 
        n_snp: int64, 
        n_insertion: int64, 
        n_deletion: int64, 
        n_transition: int64, 
       

`sample_qc`에 보면, DP, GQ, call_rate 등에 대한 여러가지 정보들이 제공되는 것을 볼 수 있습니다. 얻어진 정보는 `annotate_cols()` 함수를 이용하여, 원래 matrix table에 추가하도록 하겠습니다. 이후, `cols()`를 사용하여, 컬럼, 즉 샘플에 대한 정보만 matrix table로 추출합니다.

In [ ]:
m_qc = mt.annotate_cols(n_non_ref_SNV = mt_SNV.cols()[mt.s].sample_qc.n_non_ref,
                        n_non_ref_Indel = mt_Indel.cols()[mt.s].sample_qc.n_non_ref,
                        r_ti_tv = mt_SNV.cols()[mt.s].sample_qc.r_ti_tv,
                        dp_mean_SNV = mt_SNV.cols()[mt.s].sample_qc.dp_stats.mean,
                        dp_mean_Indel = mt_Indel.cols()[mt.s].sample_qc.dp_stats.mean,
                        gq_mean_SNV = mt_SNV.cols()[mt.s].sample_qc.gq_stats.mean,
                        gq_mean_Indel = mt_Indel.cols()[mt.s].sample_qc.gq_stats.mean,
                        call_rate_SNV = mt_SNV.cols()[mt.s].sample_qc.call_rate,
                        call_rate_Indel = mt_Indel.cols()[mt.s].sample_qc.call_rate
                     ).cols().key_by('s')

m_qc.describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Row fields:
    's': str 
    'fam': str 
    'pat_id': str 
    'mat_id': str 
    'isFemale': bool 
    'phenotype': bool 
    'ROLE': str 
    'population': str 
    'height': float64 
    'NVIQ': float64 
    'n_non_ref_SNV': int64 
    'n_non_ref_Indel': int64 
    'r_ti_tv': float64 
    'dp_mean_SNV': float64 
    'dp_mean_Indel': float64 
    'gq_mean_SNV': float64 
    'gq_mean_Indel': float64 
    'call_rate_SNV': float64 
    'call_rate_Indel': float64 
----------------------------------------
Key: ['s']
----------------------------------------


이제 샘플별로 나타난 SNV와 indel의 숫자를 확인해보도록 합시다. 아래 `non_ref`로 시작하는 것은 non-reference calls, 즉 variant를 의미하고, heterozygous variant와 alternative homozygous variant를 말합니다. SNV와 indel의 숫자를 확인하는건 전장유전체 데이터에서 가장 먼저 확인할 부분 중 하나입니다.

> 리빙포인트: SNV와 indel의 분포가 군집을 이루는 것처럼 보인다면, 1) 주어진 데이터셋에 ancestry가 다른 샘플이 존재하거나, 2) 배치효과가 존재함을 의미합니다.

In [ ]:
p = hl.plot.scatter(x=m_qc.n_non_ref_SNV,
                    y=m_qc.n_non_ref_Indel,
                    xlabel='SNV',
                    ylabel='Indel',
                    hover_fields={'ID': m_qc.s},
                    title = 'Number of variants',
                    size=8)
show(p)

몇가지 지표에 대한 플롯을 확인해봅시다

In [ ]:
p = hl.plot.histogram(m_qc['gq_mean_SNV'], legend='Mean Sample GQ - SNV')
show(p)

In [ ]:
p = hl.plot.histogram(m_qc['gq_mean_Indel'], legend='Mean Sample GQ - Indel')
show(p)

In [ ]:
p = hl.plot.histogram(m_qc['dp_mean_Indel'], legend='Mean Sample DP - Indel')
show(p)

In [ ]:
p = hl.plot.histogram(m_qc['call_rate_SNV'], legend='Call rate - SNV')
show(p)

### 유전변이

유전변이의 정도관리는 `variant_qc()` 함수를 통해 시행합니다. 샘플 수준의 정도관리인 `sample_qc()`와는 다른 기능입니다. `variant_qc()`는 하나의 variant (즉 row)에 대하여 모든 샘플에 나타나는 지표를 확인한 것이고, `sample_qc()`는 한 샘플에 대하여 모든 변이가 갖는 정보를 나타냅니다.

`variant_qc()`를 한 후 추가된 정보에 대하여 시각화를 해봅시다.

In [ ]:
mt = hl.variant_qc(mt)
mt.describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Column fields:
    's': str
    'fam': str
    'pat_id': str
    'mat_id': str
    'isFemale': bool
    'phenotype': bool
    'ROLE': str
    'population': str
    'height': float64
    'NVIQ': float64
----------------------------------------
Row fields:
    'locus': locus<GRCh38>
    'alleles': array<str>
    'rsid': str
    'qual': float64
    'filters': set<str>
    'info': struct {
        AC: array<int32>, 
        AF: array<float64>, 
        AN: int32, 
        BaseQRankSum: float64, 
        ClippingRankSum: float64, 
        DP: int32, 
        DS: bool, 
        END: int32, 
        FS: float64, 
        HaplotypeScore: float64, 
        InbreedingCoeff: float64, 
        MLEAC: array<int32>, 
        MLEAF: array<float64>, 
        MQ: float64, 
        MQ0: int32, 
        MQRankSum: float64, 
        NEGATIVE_TRAIN_SITE: bool, 
        POSITIVE_TRAIN_SITE: bool, 
     

In [ ]:
p = hl.plot.histogram(mt.variant_qc.dp_stats['mean'], legend='Mean DP at the variant level')
show(p)

In [ ]:
p = hl.plot.histogram(mt.variant_qc['call_rate'], legend='Call rate of variants')
show(p)

유전변이 수준에서 정도관리는 향후 downstream 분석에 매우 중요합니다. 분석에 쓰이는 변이가 품질의 차이에 의하여 연관성 분석에서 confounder로 작동하는 경우가 많습니다. 따라서 본인이 분석하고자 하는 전장유전체 데이터에서 어떤 품질지표가 있는지 면밀하게 확인할 필요가 있습니다.

In [ ]:
mt.variant_qc.describe()

In [ ]:
print("--> Before variant QC(call rate), the number of variants is :", mt.rows().count())
mt = mt.filter_rows((mt.variant_qc.call_rate >= 0.9))
print("--> After variant QC(call rate), the number of variants is :", mt.rows().count())

유전변이의 정도관리에 흔하게 적용하는 것 중에 하나는 low complexity region (LCR)에 발생하는 유전변이를 제거하는 것입니다. LCR은 bwa-mem을 개발한 Heng Li가 short read 시퀀싱에서 alignment가 잘 되지 않는 지점이라고 정의한 region입니다. 실제 여러분의 데이터에서 LCR 지역의 유전변이의 품질지표와 LCR 이외의 지역의 그것을 비교하면 상당한 차이가 발생합니다. 따라서 우리는 분석을 위해 LCR 지역의 변이를 제거하겠습니다.

In [ ]:
print("--> Before remove variants in LCR, the number of variants is :", mt.rows().count())
lcr_bed = hl.import_bed("data/LCR-hs38.bed", reference_genome = 'GRCh38')
mt = mt.filter_rows(hl.is_defined(lcr_bed[mt.locus]), keep=False)
print("--> After remove variants in LCR, the number of variants is :", mt.rows().count())

### 유전형

유전형 정보는 샘플 당 고유하게 존재합니다. 우리는 유전형 정보를 바탕으로 기대했던 population structure이 존재하는지 확인하고자 합니다. 만약 다른 패턴이 나오면, 유전형에 영향을 주는 어떤 이유를 생각해볼 수 있습니다. 우선 PCA를 시행해봅시다.

In [ ]:
eigenvalues, pcs, _ = hl.hwe_normalized_pca(mt.GT, k=3) # 시간 절약을 위해 k는 3으로 설정합니다

2023-11-06 23:18:45.404 Hail: INFO: Coerced sorted dataset
2023-11-06 23:19:01.513 Hail: INFO: Coerced sorted dataset
2023-11-06 23:20:03.624 Hail: INFO: hwe_normalize: found 9676 variants after filtering out monomorphic sites.
2023-11-06 23:20:59.519 Hail: INFO: Coerced sorted dataset
2023-11-06 23:22:36.112 Hail: INFO: pca: running PCA with 3 components...
2023-11-06 23:23:52.459 Hail: INFO: wrote table with 0 rows in 0 partitions to /tmp/persist_TablegBIgK2kMy4


In [ ]:
mt = mt.annotate_cols(scores = pcs[mt.s].scores)

In [ ]:
p = hl.plot.scatter(mt.scores[0],
                    mt.scores[1],
                    label=mt.population,
                    title='PCA', xlabel='PC1', ylabel='PC2')
show(p)

## Variant Annotation 와 Classification 하기

VCF 데이터는 유전변이 정보를 담고 있지만, 각 유전변이가 어떠한 유전자/단백질을 변화시키는지, 기능적으로 의미가 있을지 등등에 대한 정보를 제공하지 않습니다. 그래서 유전변이의 위치와 서열에 따라 위 정보들을 추가하는 과정이 필요합니다. 우리는 이 과정을 variant annotation이라 부릅니다. 우리말로 굳이 번역을 하면 '주석화'라고 하는데, 의미가 직접적으로 전달되진 않습니다.

Variant annotation은 유전변이 분석에서 매우 중요한 부분을 차지합니다. 변이의 기능성을 판별하는 과정이면서 동시에 annotation에 사용되는 다양한 방법과 참조 유전체 데이터 및 리소스에 따라 결과가 조금씩 다르게 나옵니다. 그래서 어떠한 방법으로 annotation을 했는지 기준을 세우고 정확히 명시할 필요가 있습니다.

우리는 튜토리얼에서 Ensembl Variant Effect Predictor(VEP)를 이용한 variant annotation을 하려고 합니다. Hail에서는 matrix table에 직접 VEP를 할 수 있게 아래와 같이 `vep()`라는 함수를 제공합니다.

```
mt_vep = hl.vep(mt, "path/to/vep-configuration.json")
```

VEP가 설치된 path와 annotation 파라미터를 json 파일로 저장하고, `vep()` 함수에 넣어주면 됩니다. 하지만 이렇게 hail 내에서 VEP annotation을 하려면 상당히 많은 리소스가 필요하기 때문에, 튜토리얼에서 진행하긴 어렵기 때문에, 제가 커맨드라인으로 미리 VEP를 했고, 처음 불러들인 VCF에 VEP annotation이 포함되어 있습니다.

```
# 터미널에서 VEP 구동하기
./vep -i $in.vcf -o $out.vcf --vcf \
  --per_gene --pick --pick_order canonical,appris,tsl,biotype,ccds,rank,length --cache
```

우리는 이번 튜토리얼에선 이 데이터를 사용하지만, 보통은 hail 내의 `vep()` 함수를 사용하길 추천합니다.


> 한가지 꼭 기억을 해야할 것은 variant annotation 방식은 전장유전체를 포함한 유전체 연구별로 상당히 상이합니다. 향후 분석 (특히 대규모 분석)에서 연구 재현성을 확보하기 위해선, variant annotation 코드와 scheme을 정확히 명시할 필요가 있습니다. 아마 많은 연구자들이 VEP 말고 ANNOVAR, SNPEFF 등을 많이 사용할겁니다. 이런 툴들도 상당히 좋습니다만, 요즘 이뤄지는 대규모 연구들은 VEP를 기반으로 annotation을 합니다. 각 툴은 고유의 annotation scheme이 있기 때문에 (예를들어, coding variant에서 어떤 순서로 consequence를 주는가? 등등), ANNOVAR의 결과와 VEP의 결과가 정확히 일치하지 않을수 있습니다. 이후 downstream 분석들 (연관성 분석 등등)에서 다른 결과를 줄수 있죠. 그래서 가능한 널리 통용되는 툴을 사용할 필요가 있고, 개인적인 추천은 VEP 입니다.


    
### VEP와 variant annotation

VEP를 이용한 variant annotation은 다음과 같은 기능을 제공합니다.

- 유전변이의 결과값(consequence): stop codon, nonsynonymous, UTR, upstream (promoter), intron, intergenic
- 유전변이의 기능적 수치(functional score): SIFT, PolyPhen2, CADD 등등
- 유전변이의 기능적 정보들: epigenome, transcriptome 데이터 등등

다양한 기능적 정보들을 plugin을 통해 제공하고 유저는 자유롭게 추가할 수 있습니다.


우리는 input VCF에 VEP를 미리 해두었기 때문에, 이에 대한 정보가 VCF header에 기록되어 있습니다. VEP는 consequence 정보를 `CSQ`라는 tag으로 기록하고, 정보들을 `|`으로 구분하여 제공합니다. 이 정보는 각 variant (row)에 동일한 순서로 제공합니다.

이제 `get_vcf_metadata()` 함수를 이용하여, CSQ에 어떤 정보가 기록되었는지 확인해봅시다.

In [ ]:
h = hl.get_vcf_metadata('data/out.test.chr22.vcf.bgz')
h['info']['CSQ']['Description']

'Consequence annotations from Ensembl VEP. Format: Allele|Consequence|IMPACT|SYMBOL|Gene|Feature_type|Feature|BIOTYPE|EXON|INTRON|HGVSc|HGVSp|cDNA_position|CDS_position|Protein_position|Amino_acids|Codons|Existing_variation|DISTANCE|STRAND|FLAGS|SYMBOL_SOURCE|HGNC_ID'

위에서 확인한 바와 같이 정보들은 array의 형태로 제공이 됩니다. 우리는 이 정보를 matrix table에 `CSQ` 필드에 저장하여 이후 분석에 사용합니다.

In [ ]:
mt = mt.annotate_rows(CSQ = hl.struct(
    Allele=mt.info.CSQ[0].split('\|')[0],
    Consequence=mt.info.CSQ[0].split('\|')[1],
    IMPACT=mt.info.CSQ[0].split('\|')[2],
    SYMBOL=mt.info.CSQ[0].split('\|')[3],
    Gene=mt.info.CSQ[0].split('\|')[4],
    DISTANCE=hl.int64(mt.info.CSQ[0].split('\|')[18]))
)
mt.describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Column fields:
    's': str
    'fam': str
    'pat_id': str
    'mat_id': str
    'isFemale': bool
    'phenotype': bool
    'ROLE': str
    'population': str
    'height': float64
    'NVIQ': float64
    'scores': array<float64>
----------------------------------------
Row fields:
    'locus': locus<GRCh38>
    'alleles': array<str>
    'rsid': str
    'qual': float64
    'filters': set<str>
    'info': struct {
        AC: array<int32>, 
        AF: array<float64>, 
        AN: int32, 
        BaseQRankSum: float64, 
        ClippingRankSum: float64, 
        DP: int32, 
        DS: bool, 
        END: int32, 
        FS: float64, 
        HaplotypeScore: float64, 
        InbreedingCoeff: float64, 
        MLEAC: array<int32>, 
        MLEAF: array<float64>, 
        MQ: float64, 
        MQ0: int32, 
        MQRankSum: float64, 
        NEGATIVE_TRAIN_SITE: bool, 
        POSI

이제 총 몇개의 consequence가 발생했는지 확인해봅시다. `counter()` 함수를 사용하여 계수를 합니다.

In [ ]:
d = mt.aggregate_rows(hl.agg.counter(mt.CSQ.Consequence))
pd.DataFrame.from_dict(d, orient='index')

2023-11-06 23:25:01.344 Hail: INFO: Coerced sorted dataset
2023-11-06 23:25:24.435 Hail: INFO: Coerced sorted dataset


,0
3_prime_UTR_variant,455
5_prime_UTR_variant,120
downstream_gene_variant,976
frameshift_variant,12
inframe_deletion,4
inframe_insertion,2
intergenic_variant,285
intron_variant,9378
missense_variant,671
missense_variant&splice_region_variant,17


예상했던대로 noncoding genome에 변이들(intronic, integenic, upstream/downstream)에 많이 발생하는 것을 볼수 있습니다.

이번에는 유전변이가 발생하는 위치에 따라 변이의 빈도수가 어떻게 달라지는지 한번 확인해봅시다.

In [ ]:
p = hl.plot.histogram(mt.filter_rows(mt.CSQ.Consequence=='intergenic_variant').info.AC[0],
                      title = 'Allele counts of intergenic variants')
show(p)

2023-11-06 23:27:33.761 Hail: INFO: Coerced sorted dataset
2023-11-06 23:27:54.616 Hail: INFO: Coerced sorted dataset
2023-11-06 23:30:06.836 Hail: INFO: Coerced sorted dataset
2023-11-06 23:30:30.120 Hail: INFO: Coerced sorted dataset


In [ ]:
p = hl.plot.histogram(mt.filter_rows(mt.CSQ.Consequence=='missense_variant').info.AC[0],
                      title = 'Allele counts of missense variants')
show(p)

2023-11-06 23:32:44.227 Hail: INFO: Coerced sorted dataset
2023-11-06 23:33:04.278 Hail: INFO: Coerced sorted dataset
2023-11-06 23:35:13.810 Hail: INFO: Coerced sorted dataset
2023-11-06 23:35:35.102 Hail: INFO: Coerced sorted dataset


Intergenic variant가 missense variant보다 high AC variant가 많은 것을 볼 수 있습니다.

### 커스텀 데이터 사용

Variant annotation은 실험에 따라 다양한 외부 데이터를 추가할 수 있습니다. 이번에는 hail table로 저장된 annotation 데이터를 추가해보겠습니다. 아래는 missense variant의 위험도를 측정하기 위해 [MPC (missense badness, PolyPhen-2, and constraint) score](https://www.biorxiv.org/content/10.1101/148353v1)를 사용해보겠습니다.

In [ ]:
MPC = hl.read_table("data/MPC_chr22.ht")
MPC.show()

,,,,,,,,,,,,,,,,,
gene_name,ENST,ENSG,CCDS,Consequence,HGVSc,HGVSp,Amino_acids,context,SIFT,PolyPhen,obs_exp,mis_badness,fitted_score,MPC,variant,locus,alleles
str,str,str,str,str,str,str,str,str,str,str,float64,float64,float64,float64,str,locus<GRCh38>,array<str>
"""TRABD""","""ENST00000303434""","""ENSG00000170638""","""CCDS14086.1""","""missense_variant&splice_region_variant""","""ENST00000303434.4:c.421A>C""","""ENSP00000305664.4:p.Asn141His""","""N/H""","""GAA""","""tolerated(0.1)""","""possibly_damaging(0.566)""",8.46e-01,3.89e-01,9.98e-01,6.99e-01,"""22:50197241:A:C""",chr22:50197241,"[""A"",""C""]"
"""TRABD""","""ENST00000303434""","""ENSG00000170638""","""CCDS14086.1""","""missense_variant&splice_region_variant""","""ENST00000303434.4:c.421A>G""","""ENSP00000305664.4:p.Asn141Asp""","""N/D""","""GAA""","""deleterious(0.04)""","""possibly_damaging(0.869)""",8.46e-01,4.86e-01,9.95e-01,1.01e+00,"""22:50197241:A:G""",chr22:50197241,"[""A"",""G""]"
"""TRABD""","""ENST00000303434""","""ENSG00000170638""","""CCDS14086.1""","""missense_variant&splice_region_variant""","""ENST00000303434.4:c.421A>T""","""ENSP00000305664.4:p.Asn141Tyr""","""N/Y""","""GAA""","""deleterious(0.01)""","""benign(0.444)""",8.46e-01,5.61e-01,9.98e-01,6.86e-01,"""22:50197241:A:T""",chr22:50197241,"[""A"",""T""]"
"""TRABD""","""ENST00000303434""","""ENSG00000170638""","""CCDS14086.1""","""missense_variant&splice_region_variant""","""ENST00000303434.4:c.422A>C""","""ENSP00000305664.4:p.Asn141Thr""","""N/T""","""AAC""","""deleterious(0.01)""","""possibly_damaging(0.869)""",8.46e-01,3.74e-01,9.95e-01,9.45e-01,"""22:50197242:A:C""",chr22:50197242,"[""A"",""C""]"
"""TRABD""","""ENST00000303434""","""ENSG00000170638""","""CCDS14086.1""","""missense_variant&splice_region_variant""","""ENST00000303434.4:c.422A>G""","""ENSP00000305664.4:p.Asn141Ser""","""N/S""","""AAC""","""tolerated(0.23)""","""benign(0.241)""",8.46e-01,1.43e-01,9.99e-01,3.72e-01,"""22:50197242:A:G""",chr22:50197242,"[""A"",""G""]"
"""TRABD""","""ENST00000303434""","""ENSG00000170638""","""CCDS14086.1""","""missense_variant&splice_region_variant""","""ENST00000303434.4:c.422A>T""","""ENSP00000305664.4:p.Asn141Ile""","""N/I""","""AAC""","""deleterious(0)""","""probably_damaging(0.921)""",8.46e-01,5.84e-01,9.93e-01,1.11e+00,"""22:50197242:A:T""",chr22:50197242,"[""A"",""T""]"
"""TRABD""","""ENST00000303434""","""ENSG00000170638""","""CCDS14086.1""","""missense_variant&splice_region_variant""","""ENST00000303434.4:c.423C>A""","""ENSP00000305664.4:p.Asn141Lys""","""N/K""","""ACG""","""deleterious(0.01)""","""possibly_damaging(0.597)""",8.46e-01,4.99e-01,9.97e-01,7.79e-01,"""22:50197243:C:A""",chr22:50197243,"[""C"",""A""]"
"""TRABD""","""ENST00000303434""","""ENSG00000170638""","""CCDS14086.1""","""missense_variant&splice_region_variant""","""ENST00000303434.4:c.423C>G""","""ENSP00000305664.4:p.Asn141Lys""","""N/K""","""ACG""","""deleterious(0.01)""","""possibly_damaging(0.597)""",8.46e-01,4.99e-01,9.97e-01,7.79e-01,"""22:50197243:C:G""",chr22:50197243,"[""C"",""G""]"


In [ ]:
MPC = MPC.key_by("locus", "alleles")
mt = mt.key_rows_by(mt.locus, mt.alleles)
mt = mt.annotate_rows(MPC = MPC[mt.locus, mt.alleles].MPC) # locus와 allele을 기준으로 두 matrix table을 연결합니다
print("--> The number of variants with MPC ≥ 2 is :", mt.filter_rows(mt.MPC >= 2).rows().count())

2023-11-06 23:37:52.856 Hail: INFO: Coerced sorted dataset
2023-11-06 23:38:13.919 Hail: INFO: Coerced sorted dataset


--> The number of variants with MPC ≥ 2 is : 3


MPC score가 잘 annotate 된 것으로 확인됩니다. MPC가 2이상인 변이를 선별해보니 총 3개가 존재합니다.

이번에는 allele frequency (AF)에 대한 public database를 사용하여, annotation을 해봅시다. 현재 VCF에는 1000 Genome Project 팀에서 미리 만들어둔 AF 정보가 존재하지만, 튜토리얼을 위해 없다고 가정하고 진행해보겠습니다. 가장 널리 사용되는 AF 데이터는 [gnomAD](https://gnomad.broadinstitute.org/)입니다. 우리는 튜토리얼을 위해, [한국인 전장유전체](https://www.science.org/doi/10.1126/sciadv.aaz7835)과 [일본인 전장유전체](https://togovar.biosciencedbc.jp) 연구에서 얻은 AF를 포함한 AF를 `AF` 컬럼에 저장했고, 이 정보를 사용하도록 하겠습니다.  

In [ ]:
AF_table = hl.read_table('data/gnomadv3_1_non_neuro_adj_chr22.ht')
AF_table.describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Row fields:
    'locus': locus<GRCh38> 
    'alleles': array<str> 
    'AC': int32 
    'AN': int32 
    'AF': float64 
    'homozygote_count': int32 
----------------------------------------
Key: ['locus', 'alleles']
----------------------------------------


위와 마찬가지로 locus와 allele을 사용하여 AF 정보를 추가하고자 합니다.

In [ ]:
AF_table = AF_table.key_by("locus", "alleles")
mt = mt.annotate_rows(public_AF = AF_table[mt.locus, mt.alleles])
# mt = mt.key_rows_by(mt.locus, mt.alleles)
mt.describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Column fields:
    's': str
    'fam': str
    'pat_id': str
    'mat_id': str
    'isFemale': bool
    'phenotype': bool
    'ROLE': str
    'population': str
    'height': float64
    'NVIQ': float64
    'scores': array<float64>
----------------------------------------
Row fields:
    'locus': locus<GRCh38>
    'alleles': array<str>
    'rsid': str
    'qual': float64
    'filters': set<str>
    'info': struct {
        AC: array<int32>, 
        AF: array<float64>, 
        AN: int32, 
        BaseQRankSum: float64, 
        ClippingRankSum: float64, 
        DP: int32, 
        DS: bool, 
        END: int32, 
        FS: float64, 
        HaplotypeScore: float64, 
        InbreedingCoeff: float64, 
        MLEAC: array<int32>, 
        MLEAF: array<float64>, 
        MQ: float64, 
        MQ0: int32, 
        MQRankSum: float64, 
        NEGATIVE_TRAIN_SITE: bool, 
        POSI

In [ ]:
mt.public_AF.show()

2023-11-06 23:40:29.808 Hail: INFO: Coerced sorted dataset
2023-11-06 23:40:46.247 Hail: INFO: Coerced sorted dataset


+----------------+------------+--------------+--------------+--------------+
| locus          | alleles    | public_AF.AC | public_AF.AN | public_AF.AF |
+----------------+------------+--------------+--------------+--------------+
| locus<GRCh38>  | array<str> |        int32 |        int32 |      float64 |
+----------------+------------+--------------+--------------+--------------+
| chr22:50197100 | ["T","C"]  |          382 |       134796 |     2.83e-03 |
| chr22:50197103 | ["C","T"]  |            9 |       134814 |     6.68e-05 |
| chr22:50197198 | ["C","T"]  |        51372 |       134586 |     3.82e-01 |
| chr22:50197199 | ["G","A"]  |           13 |       134682 |     9.65e-05 |
| chr22:50197199 | ["G","C"]  |         7859 |       134506 |     5.84e-02 |
| chr22:50197200 | ["C","G"]  |         4313 |       134758 |     3.20e-02 |
| chr22:50197205 | ["G","A"]  |          476 |       130900 |     3.64e-03 |
| chr22:50197285 | ["G","C"]  |            1 |       134834 |     7.42e-06 |
| chr22:50197300 | ["C","T"]  |          137 |       134830 |     1.02e-03 |
| chr22:50197309 | ["G","C"]  |            2 |       134816 |     1.48e-05 |
+----------------+------------+--------------+--------------+--------------+

+----------------------------+
| public_AF.homozygote_count |
+----------------------------+
|                      int32 |
+----------------------------+
|                          3 |
|                          0 |
|                       9834 |
|                          0 |
|                        868 |
|                        268 |
|                          4 |
|                          0 |
|                          0 |
|                          0 |
+----------------------------+
showing top 10 rows

## Qualifying variant 추출하기

우리가 유전변이 연관성 분석을 할때, input으로 들어가는 변이를 qualifying variant (QV)라고 부릅니다. 각 연구에서 검정하고자 하는 가설에 따라 원하는 유전변이가 다릅니다. Common variant를 분석하려면, AF를 선정하거나 LD를 고려하여 변이를 선별해야합니다. Rare recessive variant를 분석하고자 한다면, genotype에서 homozygous variant를 선별하고, 특정 인구집단에서 AF가 몇 퍼센트 이하인 기준을 세워서 필터링을 합니다. 이렇게 각 목적마다 주어지는 input이 다르고, 이를 정의하여 추출한 변이를 QV라고 부릅니다. 또한 위에서 본 것과 같이 많은 false positive를 적절하게 제어하기 위해서도 필터링 단계는 매우 중요합니다.

이번 섹션에서는 위에서 연습한 내용을 종합하여, 목적에 맞는 QV를 선별해봅시다.



### 신규변이 (de novo variant)

De novo variant (DNV)는 발달성 질환에 가장 주된 원인으로 지난 10년간 NGS연구에서 암의 somatic mutation과 함께 가장 널리 연구된 주제입니다. DNV를 추출하는 여러가지 알고리즘이 있지만, VCF를 외부 프로그램 사용을 위해 변환하고 다시 수집하는 과정은 코드의 재현성이나 tidy함을 확보하기 굉장히 어렵습니다. 그래서 Hail은 DNV 탐색 알고리즘을 제공하고 있습니다. 우리는 대규모 자폐증 전장유전체 연구([An et al. 2018, Science](https://www.science.org/doi/10.1126/science.aat6576))의 필터링 기준을 따라 DNV를 추출해보고자 합니다.

* 선별 기준
     - AF ≤ 0.001 (0.1%)
     - Filtering high quality de novo variants using quality metrics(AB, DP, GQ)
     - DNV test high/medium confidence
     


In [ ]:
DNV = mt.filter_rows(mt.public_AF.AF <= 0.001) # 0.1%
DNV.count()

2023-11-06 23:42:27.865 Hail: INFO: Coerced sorted dataset
2023-11-06 23:42:44.669 Hail: INFO: Coerced sorted dataset


(10759, 1784)

In [ ]:
trio_fam = hl.Pedigree.read('data/1kg_v3_20200731_complete_fam_633.ped')
dnv_test = hl.de_novo(mt=DNV,
                        pedigree=trio_fam,
                        pop_frequency_prior=DNV.public_AF.AF,
                        max_parent_ab=0.05,
                        min_child_ab = 0.3,
                        min_dp_ratio = 0.3,
                        min_gq=90)
print("--> De novo variants from complete trio(n_proband+control = 596) : ", dnv_test.aggregate(hl.agg.counter(dnv_test.confidence)))

2023-11-06 23:45:36.189 Hail: INFO: Coerced sorted dataset
2023-11-06 23:45:53.623 Hail: INFO: Coerced sorted dataset


--> De novo variants from complete trio(n_proband+control = 596) :  {'HIGH': 10, 'LOW': 94, 'MEDIUM': 7}


In [ ]:
DNV = dnv_test.filter((dnv_test.confidence=="HIGH") | (dnv_test.confidence=="MEDIUM"))
DNV.count()

2023-11-06 23:49:37.716 Hail: INFO: Coerced sorted dataset
2023-11-06 23:49:54.755 Hail: INFO: Coerced sorted dataset


17

### 희귀변이 (Rare Variants)

희귀변이는 집단에 AF가 낮게 존재하는 변이를 말합니다. 전장유전체 코호트가 커질수록 질환이나 형질에 영향을 보다 큰 영향을 미칠 것이라 생각하는 희귀변이 연구들이 많이 진행되고 있습니다. 희귀변이는 공통변이(common variant)와 다르게 품질지표나 AF 데이터베이스에 따른 분포가 상이하기 때문에 QV를 추출할때 보다 정확하게 진행을 해야합니다. 우리는 대규모 유전체 연구([Satterstrom et al. 2020 Cell](https://www.cell.com/cell/fulltext/S0092-8674(19)31398-4), [Werling et al. 2018 Nature Genetics](https://www.nature.com/articles/s41588-018-0107-y))에서 시행된 추출 방법에 따라 heterozygous와 homozygous variant를 추출해보고자 합니다.

#### Rare heterozygous variant


* 선별 기준
     - AF ≤ 0.001, AC ≤ 5
     - Filtering high quality heterozygous variants using quality metrics(GQ, AB, VQSLOD)


In [ ]:
filter_condition_entry = ((mt.GT.is_het()) & (mt.GQ >= 25) & (mt.AB >= 0.3))
RareHet = mt.filter_entries(filter_condition_entry)
print("--> After remove variants with low quality(based on GQ, AB), the number of het entries is :", RareHet.entries().count())

2023-11-06 23:53:02.640 Hail: INFO: Coerced sorted dataset
2023-11-06 23:53:18.982 Hail: INFO: Coerced sorted dataset


--> After remove variants with low quality(based on GQ, AB), the number of het entries is : 646006


In [ ]:
filter_condition_variant = (((hl.is_snp(RareHet.alleles[0], RareHet.alleles[1])) & (RareHet.info.VQSLOD >= -2.085))|
                            (~hl.is_snp(RareHet.alleles[0], RareHet.alleles[1])))
RareHet = RareHet.filter_rows(filter_condition_variant)
print("--> After remove SNVs with VQSLOD<-2.085, the number of variants is reduced to :", RareHet.rows().count())

2023-11-06 23:55:33.582 Hail: INFO: Coerced sorted dataset
2023-11-06 23:55:49.722 Hail: INFO: Coerced sorted dataset


--> After remove SNVs with VQSLOD<-2.085, the number of variants is reduced to : 13284


In [ ]:
RareHet = RareHet.filter_rows((RareHet.public_AF.AF<=0.001) & (RareHet.info.AC[0]<=5))
RareHet.count()

2023-11-06 23:58:04.805 Hail: INFO: Coerced sorted dataset
2023-11-06 23:58:21.312 Hail: INFO: Coerced sorted dataset


(8639, 1784)

In [ ]:
RareHet = RareHet.filter_cols(hl.is_defined(RareHet.ROLE)) # 자녀에게 있는 heterozygous만 선별
RareHet = RareHet.entries()
print("--> In 596 children, there are ", RareHet.count(), " high quality rare het mutations.")

2023-11-07 00:00:56.070 Hail: INFO: Coerced sorted dataset
2023-11-07 00:01:12.750 Hail: INFO: Coerced sorted dataset


--> In 596 children, there are  2451  high quality rare het mutations.


#### Rare Homozygous variant

Homozygous variant는 genotype을 갖고 필터링을 합니다. 더 복잡한 상황을 가정해보죠. 부모에게는 heterozygous variant이고, 자녀에게는 homozygous variant로 나오는 경우들을 추출해봅시다.

* 선별기준
     - Filtering homozygous variants where genotype of both parents are heterozygous
     - Filtering high quality homozygous variants using quality metrics(GQ, AB, DP, AN, QD)

In [ ]:
RareHom = mt.filter_entries(mt.GT.is_non_ref())
RareHom = RareHom.filter_rows(RareHom.public_AF.AF <= 0.05)

In [ ]:
tb = RareHom.key_cols_by().entries()
RareHom = RareHom.unpersist()
rare_hom = tb.filter((hl.is_missing(tb.ROLE)) & (tb.GT.is_het_ref()))
rare_hom_tag = rare_hom.group_by(rare_hom.locus, rare_hom.alleles, rare_hom.fam).aggregate(nHet = hl.agg.collect(rare_hom.ROLE))
rare_hom_tag = rare_hom_tag.annotate(nHet = hl.len(rare_hom_tag.nHet))
rare_hom_tag = rare_hom_tag.filter(rare_hom_tag.nHet == 2)
tb = tb.key_by('locus', 'alleles', 'fam').semi_join(rare_hom_tag)
print("--> After variant filtering families with het parents, the number of variants is :", tb.count())

2023-11-07 00:03:32.514 Hail: INFO: Coerced sorted dataset
2023-11-07 00:03:49.143 Hail: INFO: Coerced sorted dataset
2023-11-07 00:04:59.958 Hail: INFO: Ordering unsorted dataset with network shuffle
2023-11-07 00:05:56.591 Hail: INFO: Coerced sorted dataset
2023-11-07 00:06:13.708 Hail: INFO: Coerced sorted dataset
2023-11-07 00:07:21.191 Hail: INFO: Ordering unsorted dataset with network shuffle


--> After variant filtering families with het parents, the number of variants is : 59252


In [ ]:
filter_condition = ((tb.GT.is_hom_var()) & (tb.GQ >= 30) & (tb.AB >= 0.95) & (tb.DP >= 18))
tb = tb.filter(filter_condition)
print("--> After remove variants with low quality(based on GQ, AB, DP), the number of hom entries is :", tb.count())

2023-11-07 00:11:03.401 Hail: INFO: Coerced sorted dataset
2023-11-07 00:11:30.910 Hail: INFO: Coerced sorted dataset
2023-11-07 00:13:16.598 Hail: INFO: Ordering unsorted dataset with network shuffle
2023-11-07 00:14:21.488 Hail: INFO: Coerced sorted dataset
2023-11-07 00:14:45.466 Hail: INFO: Coerced sorted dataset
2023-11-07 00:15:55.777 Hail: INFO: Ordering unsorted dataset with network shuffle


--> After remove variants with low quality(based on GQ, AB, DP), the number of hom entries is : 2884


In [ ]:
# 자녀에게 나오는 변이만 추출
tb = tb.filter(hl.is_defined(tb.ROLE))
RareHom = tb.key_by('locus', 'alleles', 's')
print("--> In 596 children, there are ", tb.count(), " high quality rare hom mutations.")

2023-11-07 00:19:53.905 Hail: INFO: Coerced sorted dataset
2023-11-07 00:20:18.981 Hail: INFO: Coerced sorted dataset
2023-11-07 00:22:01.144 Hail: INFO: Ordering unsorted dataset with network shuffle
2023-11-07 00:22:59.617 Hail: INFO: Coerced sorted dataset
2023-11-07 00:23:22.881 Hail: INFO: Coerced sorted dataset
2023-11-07 00:24:46.723 Hail: INFO: Ordering unsorted dataset with network shuffle


--> In 596 children, there are  2884  high quality rare hom mutations.


## Association testing

Hail을 이용하여서 [GWAS](https://hail.is/docs/0.2/tutorials/01-genome-wide-association-study.html#GWAS-Tutorial), [SKAT](https://hail.is/docs/0.2/methods/genetics.html#hail.methods.skat) 등의 다양한 연관성 방법을 사용할 수 있습니다. 추가 튜토리얼은 위 링크를 참고하길 바랍니다.

### Linear regression

Hail을 이용하여 간단한 regression 검정을 해봅시다. Sample 정보에 있는 임의의 정보를 사용합니다.


In [ ]:
result_ht = hl.linear_regression_rows(y=mt.height,
                                x=mt.GT.n_alt_alleles(),
                                covariates=[1, mt.isFemale, mt.scores[0], mt.scores[1]])


ConnectionError: ignored

In [ ]:
result_ht.show()